In [1]:
from wsi.indicators.education import build_education_df

edu_world = build_education_df()

In [2]:
edu_world

,Year,ISO_code,Education
0,1971,AUS,0.849095
1,2004,AUS,0.973436
2,2005,AUS,0.976048
3,2006,AUS,0.985507
4,2007,AUS,0.986429
...,...,...,...
1508,2022,COM,0.744584
1509,2023,YEM,0.485030
1510,2019,NCL,0.986383
1511,2021,NIC,0.994113


In [3]:
from wsi.utils import raw_data_path
import pandas as pd

df = pd.read_csv(raw_data_path("NATMON_DS_27102024204132612.csv"))

In [4]:
df

,NATMON_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",AUS,Australia,1971,1971,8.44000,NaN,NaN
1,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",AUS,Australia,2004,2004,11.36000,NaN,NaN
2,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",AUS,Australia,2005,2005,11.41000,NaN,NaN
3,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",AUS,Australia,2006,2006,11.56000,NaN,NaN
4,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",AUS,Australia,2007,2007,11.63000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6065,MYS_1T8_AG25T99_M,"Mean years of schooling (ISCED 1 or higher), p...",YEM,Yemen,2023,2023,7.45416,NaN,NaN
6066,MYS_1T8_AG25T99_M,"Mean years of schooling (ISCED 1 or higher), p...",ATG,Antigua and Barbuda,2021,2021,11.07553,NaN,NaN
6067,MYS_1T8_AG25T99_F,"Mean years of schooling (ISCED 1 or higher), p...",ATG,Antigua and Barbuda,2021,2021,12.17525,NaN,NaN
6068,MYS_1T8_AG25T99_GPIA,"Mean years of schooling (ISCED 1 or higher), p...",NIC,Nicaragua,2021,2021,0.99411,NaN,NaN
